<a href="https://colab.research.google.com/github/jhphan/ML-Notebooks/blob/main/tcga-ov-ml-therapy-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISB-CGC Machine Learning Notebooks
Check out other notebooks at our [Community Notebooks Repository](https://github.com/isb-cgc/Community-Notebooks)!

- **Title:** Building a simple gene expression-based classifier
- **Author:** John Phan
- **Created:** 2021-07-07
- **Purpose:** Demonstrate a basic machine learning method to predict a cancer endpoint using gene expression data.
- **URL:** https://github.com/isb-cgc/Community-Notebooks
- **Note1:** This example is based on the work published by [Bosquet et al.](https://molecular-cancer.biomedcentral.com/articles/10.1186/s12943-016-0548-9)


This notebook demonstrates how to build a basic machine learning model to predict ovarian cancer treatment outcome. Ovarian cancer gene expression data is pulled from a BigQuery table and formatted using Pandas. The data is then split into training and testing sets to build and test a logistic regression classifier using scikit-learn. 

## Import Dependencies

In [1]:
# GCP libraries
from google.cloud import bigquery
from google.colab import auth

# Pandas
import pandas as pd

# Machine learning
from sklearn.preprocessing import StandardScaler



## Authenticate

Before using BigQuery, we need to get authorized for access to BigQuery and the Google Cloud. For more information see ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html). Alternative authentication methods can be found [here](https://googleapis.dev/python/google-api-core/latest/auth.html).

In [2]:
# if you're using Google Colab, authenticate to gcloud with the following
auth.authenticate_user()

# alternatively, use the gcloud SDK
#!gcloud auth application-default login

## Parameters

Customize the following parameters based on your notebook, execution environment, or project.

In [3]:
# set the google project that will be billed for this notebook's computations
google_project = 'cgc-05-0051'

# in this example, we'll be using the Ovarian cancer TCGA dataset
cancer_type = 'TCGA-OV'

# gene expression data will be pulled from this BigQuery project
bq_project = 'isb-cgc-bq'



## BigQuery Client

Create the BigQuery client

In [4]:
# Create a client to access the data within BigQuery
client = bigquery.Client(google_project)

## Get Gene Expression Data from Big Query Table

Pull RNA-seq gene expression data from the TCGA RNA-seq BigQuery table and join it with the clinical data table to create a labeled data frame. In this example, we will label the samples based on therapy outcome. "Complete Remission/Response" will be labeled as "1" while all other therapy outcomes will be labeled as "0". This will prepare the data to build a binary classifier. 

In [5]:
ge_data = client.query(("""
  SELECT
    ge.case_barcode AS sample,
    labels.response_label AS label,
    ge.gene_name AS gene_name,
    -- Multiple samples may exist per case, take the max value
    MAX(LOG(ge.HTSeq__FPKM_UQ+1)) AS gene_expression
  FROM `{}.TCGA.RNAseq_hg38_gdc_current` AS ge
  INNER JOIN (
    SELECT
      *
    FROM (
      SELECT
        case_barcode,
        primary_therapy_outcome_success,
        CASE
          -- Complete Reponse    --> label as 1
          -- All other responses --> label as 0
          WHEN primary_therapy_outcome_success = 'Complete Remission/Response' THEN 1
          WHEN (
            primary_therapy_outcome_success IN (
              'Partial Remission/Response','Progressive Disease','Stable Disease'
            )
          ) THEN 0
        END AS response_label
        FROM `{}.TCGA_versioned.clinical_gdc_2019_06`
        WHERE
          project_short_name = '{}'
          AND primary_therapy_outcome_success IS NOT NULL
    )
  ) labels
  ON labels.case_barcode = ge.case_barcode
  WHERE gene_name IN ( -- 33 Gene signature, leave out PRSS2 (aka TRYP2)
    'RHOT1','MYO7A','ZBTB10','MATK','ST18','RPS23','GCNT1','DROSHA','NUAK1','CCPG1',
    'PDGFD','KLRAP1','MTAP','RNF13','THBS1','MLX','FAP','TIMP3','PRSS1','SLC7A11',
    'OLFML3','RPS20','MCM5','POLE','STEAP4','LRRC8D','WBP1L','ENTPD5','SYNE1','DPT',
    'COPZ2','TRIO','PDPR'
  )
  GROUP BY sample, label, gene_name
""").format(bq_project, bq_project, cancer_type)).result().to_dataframe()

print(ge_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8712 entries, 0 to 8711
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sample           8712 non-null   object 
 1   label            8712 non-null   int64  
 2   gene_name        8712 non-null   object 
 3   gene_expression  8712 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 272.4+ KB
None


## Format the Data

The data pulled from BigQuery is formatted such that each row corresponds to a sample/gene combination. However, to use the data with scikit-learn to create a prediction model, we'll need to reshape the data such that each row corresponds to a sample and each column corresponds to a gene. We'll use Pandas to pivot the data as follows.

In [6]:
ge_data_pivot = ge_data.pivot(index=('sample', 'label'), columns='gene_name', values='gene_expression').reset_index(level=['sample','label'])
print(ge_data_pivot.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sample   264 non-null    object 
 1   label    264 non-null    int64  
 2   CCPG1    264 non-null    float64
 3   COPZ2    264 non-null    float64
 4   DPT      264 non-null    float64
 5   DROSHA   264 non-null    float64
 6   ENTPD5   264 non-null    float64
 7   FAP      264 non-null    float64
 8   GCNT1    264 non-null    float64
 9   KLRAP1   264 non-null    float64
 10  LRRC8D   264 non-null    float64
 11  MATK     264 non-null    float64
 12  MCM5     264 non-null    float64
 13  MLX      264 non-null    float64
 14  MTAP     264 non-null    float64
 15  MYO7A    264 non-null    float64
 16  NUAK1    264 non-null    float64
 17  OLFML3   264 non-null    float64
 18  PDGFD    264 non-null    float64
 19  PDPR     264 non-null    float64
 20  POLE     264 non-null    float64
 21  PRSS1    264 non

In [ ]:
# remove sample names from table
ge_data_pivot_nosample = ge_data_pivot.drop(labels='sample',axis=1)
#print(ge_data_pivot_nosample.info())
#print(ge_data_pivot_nosample)

# split data into train and test sets
train_data = ge_data_pivot_nosample.sample(frac=0.5, random_state=1).sort_index()
#print(train_data.info())
#print(train_data)

test_data = ge_data_pivot_nosample.drop(train_data.index)
#print(test_data)
#print(test_data.info())

data = dict()
data['train_y'] = train_data.pop('label')
data['test_y'] = test_data.pop('label')

#scaler = StandardScaler()
data['train_x'] = scaler.fit_transform(train_data)
data['test_x'] = scaler.transform(test_data)
#data['scaler'] = scaler

#print(data['train_y'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

lr = LogisticRegression(max_iter=1000)
lr.fit(data['train_x'], data['train_y'])
pred = lr.decision_function(data['test_x'])
fpr, tpr, thresholds = metrics.roc_curve(data['test_y'], pred)
auc = metrics.auc(fpr, tpr)
print('auc:', auc)
print('auc:', auc, 'fpr:', fpr, 'tpr:', tpr, 'thresh:', thresholds)

rf = RandomForestClassifier()
rf.fit(data['train_x'], data['train_y'])
pred = rf.predict(data['test_x'])
pred_proba = rf.predict_proba(data['test_x'])
print('pred:', pred)
print('pred proba:', pred_proba)
fpr, tpr, thresholds = metrics.roc_curve(data['test_y'], pred_proba[:,1])
auc = metrics.auc(fpr, tpr)
print('auc:', auc, 'fpr:', fpr, 'tpr:', tpr, 'thresh:', thresholds)


In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

sv = svm.SVC(gamma=0.001, C=100)
sv.fit(data['train_x'], data['train_y'])
pred = sv.decision_function(data['test_x'])
print(pred)
fpr, tpr, thresholds = metrics.roc_curve(data['test_y'], pred)
auc = metrics.auc(fpr, tpr)
print('auc:', auc, 'fpr:', fpr, 'tpr:', tpr, 'thresh:', thresholds)


[-0.32458632 -2.25811669 -0.53890999 -0.09372597  0.29517058  1.64919899
 -0.04412576  0.92508798 -0.62389982 -0.44696897  2.32819176  2.7515359
  2.15020502  1.25747519  0.37994827  1.45750523 -1.03151382  1.54356784
  1.970623    0.46554394  0.40344879 -0.38878075  0.25826014  1.97900549
  0.15736329  0.11156121 -1.34952207  1.7965553   0.72278105  0.10058439
  1.20205524  0.15042829  0.82575615  3.38889774  1.702792   -0.20991142
  0.62101711  1.86964192  1.87268517  2.00842552 -0.39962761  0.92440571
 -1.27520536  2.14979576  1.69527717  1.06624378  1.37536708  1.49912412
 -0.48296274 -0.19881182 -0.45136152  0.64240058  1.89685762  0.22409761
  1.1783178   1.80010785 -0.2099434   1.29974703 -0.67821068  2.51449502
  1.03083066  1.02654118  0.620451   -0.03420452  0.40138476 -0.26879511
  0.14769651 -0.68460086  1.39902904  1.33888282  0.12831301  0.98755342
  0.63141426 -1.20705895  2.5390692  -1.01900279  0.25116914  0.52493528
  0.18561978  0.87326979  0.93744399  0.64699071  0.

In [ ]:
data['val_y'], pred

In [ ]:
# build DNN model

from keras.layers import Input, Dense, Dropout
from keras.models import Model

input_features = data['train_x'].shape[1]

# build the network
inputs = Input(shape=(input_features,), name='input')
x = Dense(64, activation='relu', name='hidden1', kernel_regularizer='l2')(inputs)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu', name='hidden2', kernel_regularizer='l2')(x)
x = Dense(16, activation='relu', name='hidden3')(x)
x = Dense(8, activation='relu', name='hidden4')(x)
x = Dense(4, activation='relu', name='hidden5')(x)
prediction = Dense(1, activation='sigmoid', name='final')(inputs)
model = Model(inputs=inputs, outputs=prediction)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
model.fit(x=data['train_x'], y=data['train_y'], \
          batch_size=32, epochs=300, verbose=1, validation_data=(data['val_x'], data['val_y']))